## 使用的package

In [1]:
from pyspark import SparkConf, SparkContext
from scipy.spatial import distance
from scipy.linalg import norm

## Map/Reduce function

這份程式碼中與Kmeans_Euclidean不一樣的地方只有在initial_clustering_map()與k_means_map()中以曼哈頓距離計算loss function與衡量資料間的距離。

In [2]:
def readfile(line):
    wordlist=line.split("\n")
    maplist=[]
    for item in wordlist:
        s = item.split(" ")
        data=[]
        for item in s:
            a=float(item)
            data.append(a)
        maplist.append((norm(data),data)) # add to map
    return maplist

In [3]:
def initial_clustering_map(x):
    d= []
    for center in x[2]:
        dis = distance.cityblock(x[1], center)
        d.append(dis)
    cluster = d.index(min(d))
    return(x[0], (cluster, x[1], x[2][cluster], min(d)))

In [4]:
def k_means_map(x):
    d = []
    for center in x[2]:
        dis = distance.cityblock(x[1][1], center)**2
        d.append(dis)
    
    min_d = min(d)
    cluster = d.index(min_d)
    if (min_d < x[1][3]):
        return (x[0], (cluster, x[1][1], x[2][cluster], min_d))
    else:
        return (x[0], x[1])

In [5]:
def new_centroid_mapper(x):
    z=[]
    for component in x[1]:
        z.append(component/num_cluster_dict[x[0]])
    return (x[0], z)

In [6]:
sc.stop()

In [7]:
conf = SparkConf().setMaster("local").setAppName("wordcount") # call sparkconf
conf = SparkConf().set("spark.default.parallelism", 4)
sc = SparkContext(conf=conf) # call sparkcontext

## Initialize

In [27]:
dataset_intial = sc.textFile("data.txt").flatMap(readfile) # read txt file [||p_k||, p_k]

In [28]:
c1 = sc.textFile("c1.txt").flatMap(readfile) # read txt file
c1 = c1.map(lambda x: x[1])
c1 = c1.collect() # list of centroids [c_1, c_2, ..., c_10]

In [29]:
loss_function = [] # value of loss function in each iteration
cluster_dict = [] # number of points in each cluster in each iteration

In [30]:
dataset = dataset_intial.map(lambda x: [x[0], x[1], c1]) # [||p_k||, p_k, [c_1, c_2, ..., c_10]]
dataset = dataset.map(initial_clustering_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)], m為cluster編號            
R = dataset

## Loop

In [31]:
for i in range (1,21): # 20次要設21
    
    # 將新的c接回去
    R = R.map(lambda x: [x[0], x[1], c1]) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||), [c_1, c_2, ..., c_10]]
    
    # 重新分群
    R = R.map(k_means_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)]
    D = R.map(lambda x: (x[1][0], x[1][3])) # (m, ||p_k-c_m||)
    
    loss = D.values().sum()
    loss_function.append(loss)
    
    new_centroid = R.map(lambda x: (x[1][0], x[1][1])) # (m, p_k)
    num_cluster_dict = new_centroid.countByKey()
    cluster_dict.append(num_cluster_dict)
    
    new_centroid = new_centroid.reduceByKey(lambda x, y: [t1+t2 for t1, t2 in zip(x, y)]) # element-wise adding
    new_centroid = new_centroid.map(new_centroid_mapper) # (m, c_m)
    c1 = new_centroid.values().collect() # list of centroids [c_1, c_2, ..., c_10]

## Case of c2.txt

In [23]:
def new_centroid_mapper2(x):
    z=[]
    for component in x[1]:
        z.append(component/num_cluster_dict2[x[0]])
    return (x[0], z)

In [21]:
c2 = sc.textFile("c2.txt").flatMap(readfile) # read txt file
c2 = c2.map(lambda x: x[1])
c2 = c2.collect() # list of centroids [c_1, c_2, ..., c_10]
loss_function2 = [] # value of loss function in each iteration
cluster_dict2 = [] # number of points in each cluster in each iteration
dataset = dataset_intial.map(lambda x: [x[0], x[1], c2]) # [||p_k||, p_k, [c_1, c_2, ..., c_10]]
dataset = dataset.map(initial_clustering_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)], m為cluster編號            
R = dataset

In [24]:
for i in range (1,21): # 20次要設21
    
    # 將新的c接回去
    R = R.map(lambda x: [x[0], x[1], c2]) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||), [c_1, c_2, ..., c_10]]
    
    # 重新分群
    R = R.map(k_means_map) # [||p_k||, (m, p_k, c_m, ||p_k-c_m||)]
    D = R.map(lambda x: (x[1][0], x[1][3])) # (m, ||p_k-c_m||)
    
    loss = D.values().sum()
    loss_function2.append(loss)
    
    new_centroid = R.map(lambda x: (x[1][0], x[1][1])) # (m, p_k)
    num_cluster_dict2 = new_centroid.countByKey()
    cluster_dict2.append(num_cluster_dict2)
    
    new_centroid = new_centroid.reduceByKey(lambda x, y: [t1+t2 for t1, t2 in zip(x, y)]) # element-wise adding
    new_centroid = new_centroid.map(new_centroid_mapper2) # (m, c_m)
    c2 = new_centroid.values().collect() # list of centroids [c_1, c_2, ..., c_10]

In [32]:
print(loss_function)

[550117.1419999995, 464829.2684039448, 470934.15384668094, 483874.81628509343, 489234.2347883463, 487664.6926267904, 483718.6659285149, 475337.9476330566, 474871.96654965664, 457244.7897417528, 447493.1956040521, 450891.8358047706, 451232.57747569657, 451860.12588546576, 451567.2235891488, 452710.05209994374, 453078.22696184996, 450646.1355620941, 450419.9701134367, 449009.59037188545]


In [25]:
print(loss_function2)

[1433739.2192009955, 1433739.2192009955, 1084488.7769648773, 973431.7146620404, 895934.5925630709, 865128.3352940814, 845846.647031348, 827219.5827561249, 803590.3456011118, 756039.5172761207, 717332.9025432297, 694587.9252526882, 684444.5019967903, 674574.7475478561, 667409.4699160281, 663556.6278215044, 660162.7772287563, 656041.3222947121, 653036.7540731612, 651112.4262522729, 649689.0131843555]


In [34]:
import csv

# 開啟輸出的 CSV 檔案
with open('loss_function_cityblock.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(loss_function)
    writer.writerow(loss_function2)

In [37]:
eucl_of_c1 = [] # 利用曼哈頓距離計算loss function時，歐式距離下，c1中centroid彼此的距離
for i in range(0,10):
    eucl_of_c1.append([])
    for j in range(0,10):
        eucl_of_c1[i].append(distance.euclidean(c1[i],c1[j]))

eucl_of_c2 = [] # 利用曼哈頓距離計算loss function時，歐式距離下，c2中centroid彼此的距離
for i in range(0,10):
    eucl_of_c2.append([])
    for j in range(0,10):
        eucl_of_c2[i].append(distance.euclidean(c2[i],c2[j]))
        
manh_of_c1 = [] # 利用曼哈頓距離計算loss function時，曼哈頓距離下，c1中centroid彼此的距離
for i in range(0,10):
    manh_of_c1.append([])
    for j in range(0,10):
        manh_of_c1[i].append(distance.cityblock(c1[i],c1[j]))

manh_of_c2 = [] # 利用曼哈頓距離計算loss function時，曼哈頓距離下，c2中centroid彼此的距離
for i in range(0,10):
    manh_of_c2.append([])
    for j in range(0,10):
        manh_of_c2[i].append(distance.cityblock(c2[i],c2[j]))

In [38]:
import csv

with open('manh_c1_euclidean.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(eucl_of_c1)

with open('manh_c2_euclidean.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(eucl_of_c2)
    
with open('manh_c1_manh.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(manh_of_c1)

with open('manh_c2_manh.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(manh_of_c2)